In [ ]:
import uuid
from typing import Dict
from datetime import datetime
from pydantic import BaseModel
from langchain.memory import ConversationBufferMemory
from langchain.schema import messages_from_dict, messages_to_dict
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()

# groq llm client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [ ]:
def groq_chat(messages: list) -> str:

    # if SessionId.session_id not in session_state:
    
    # Create a simple but context-aware prompt
    prompt = f"Answer the following user query clearly and concisely:\n\n{messages}"

    # Send request to Groq model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return response text
    return response.choices[0].message.content.strip()

In [ ]:
groq_chat(messages=["what is the capital of India?"])

'The capital of India is New Delhi.'

In [ ]:
class ChatRequest(BaseModel):
    query: str

class SessionId(BaseModel):
    session_id: str = None  # Optional, if not provided, create new

class ChatResponse(BaseModel):
    response: str

In [ ]:
# --- Memory Management ---
# In-memory active sessions
active_sessions: Dict[str, ConversationBufferMemory] = {}

In [ ]:
active_sessions

{}

In [ ]:
session_id = "a33ba0aa256c48bea03793e70b733113,"


if session_id in active_sessions.values():
    print(session_id)
else:
    print("created sessison id")

created sessison id


In [ ]:
active_sessions

{'fda9441bb78e46a79398ff0095cc8593': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 'cc5ad13eb5e5424294d933ec10d9d86e': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 'a33ba0aa256c48bea03793e70b733113': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [ ]:
# filter session id
def filter_session_id(state: SessionId) -> SessionId:
    
    if state.session_id in active_sessions.values():
        # Create new session
        state['session_id'] = state.sesstion_id
        
    return state

filter_session_id(SessionId(session_id="fda9441bb78e46a79398ff0095cc8593"))

SessionId(session_id='fda9441bb78e46a79398ff0095cc8593')

In [ ]:
# --- Session filter/create function ---
def filter_or_create_session_id(state: SessionId) -> str:
    """
    Ensures session_id exists in active_sessions.
    If not, creates a new one with ConversationBufferMemory.
    """
    session_id = state.session_id

    if not session_id or session_id not in active_sessions:
        session_id = str(uuid.uuid4().hex)
        active_sessions[session_id] = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        print(f"🆕 Created new session: {session_id}")
    else:
        print(f"✅ Using existing session: {session_id}")

    return session_id

SessionId(session_id='fda9441bb78e46a79398ff0095cc8593')

In [ ]:
def filter_or_create_session_id(state: SessionId):
    
    session_id = state.session_id

    if session_id not in active_sessions:
        # Create new session
        state.session_id = str(uuid.uuid4().hex)
        active_sessions[state.session_id] = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    else:
        # Use existing session
        return state.session_id
    
# filter or create sesison id
filtered_session_id = filter_or_create_session_id(SessionId(session_id="fda9441bb78e46a79398ff0095cc8593"))
print(filtered_session_id)

fda9441bb78e46a79398ff0095cc8593


In [ ]:
active_sessions

{'fda9441bb78e46a79398ff0095cc8593': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 'cc5ad13eb5e5424294d933ec10d9d86e': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 'a33ba0aa256c48bea03793e70b733113': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 '99f4ddfb1ef74a7eab5077c4c4ca8d5f': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [ ]:
session_id = "fda9441bb78e46a79398ff0095cc8593"

if filter_or_create_session_id(SessionId(session_id=session_id)):
    print(session_id)
else:
    print("created session id")

fda9441bb78e46a79398ff0095cc8593


In [ ]:
def chatbot(state: ChatRequest) -> ChatResponse:

    # user message
    query = state.query
    
    # Create a simple but context-aware prompt
    prompt = f"Answer the following user query clearly and concisely:\n\n{query}"

    # Send request to Groq model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return response text
    response_text = response.choices[0].message.content.strip()
    return ChatResponse(response=response_text)

In [ ]:
response = chatbot(ChatRequest(query="what is the capital of India?")).response
print(response)

The capital of India is New Delhi.


<!-- ### Now manage the session with memeory in chatbot -->

In [ ]:
active_sessions

{'fda9441bb78e46a79398ff0095cc8593': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 'cc5ad13eb5e5424294d933ec10d9d86e': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 'a33ba0aa256c48bea03793e70b733113': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 '99f4ddfb1ef74a7eab5077c4c4ca8d5f': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [ ]:
session_id = "fda9441bb78e46a79398ff0095cc8593"

if filter_or_create_session_id(SessionId(session_id=session_id)):
    print(session_id)
else:
    print("created session id")

In [ ]:
query: ChatRequest = "hello"

In [ ]:
query

'hello'

In [ ]:
# --- Chatbot function ---
def chat_bot(request: ChatRequest, save_after_chat: bool = False) -> ChatResponse:

    # --- Load or create session ---
    if filter_or_create_session_id(SessionId(session_id=session_id)) in active_sessions:
        session_id = active_sessions[session_id]
        memory = ConversationBufferMemory(return_messages=True)
        if not memory:
            # Load from disk if available
            memory = load_session(session_id)

    # --- Add user message ---
    memory.chat_memory.add_user_message(request.message)

    # --- Prepare messages for Groq ---
    messages_list = [
        {"role": "user" if msg.type == "human" else "assistant", "content": msg.content}
        for msg in memory.chat_memory.messages
    ]

    # --- Get assistant response ---
    assistant_reply = groq_chat(messages_list)
    memory.chat_memory.add_ai_message(assistant_reply)

    # --- Update active sessions ---
    active_sessions[session_id] = memory

    # --- Optionally save session to disk ---
    if save_after_chat:
        save_session(session_id, memory)
        active_sessions.pop(session_id, None)  # remove from active sessions

    return ChatResponse(session_id=session_id, history=messages_list + [{"role": "assistant", "content": assistant_reply}])

In [ ]:
def get_response(query: str, session_id: str) -> str:

    session_id = get_session_id(SessionId(session_id=session_id))

    

    response = chatbot(ChatRequest(query=query)).response
    return response

In [ ]:
def get_response(query) -> str:
    response = chatbot(ChatRequest(query=query)).response
    return response

In [ ]:
get_response(query="what is the capital of India?")

'The capital of India is New Delhi.'